# What?

**Stand:** 27/06/17

Bestimmt die Intervalle in welchen sich das Kamerasetup nicht bewegt hat.

Der Anfang und Ende eines Intervalls wird über die mittels ECC bestimmte Translation festgestellt. Liegt die Distanz einer Translation über einem bestimmten Schwellwert, wird dies als Veränderung des Kamerasetups gewertet.

In [1]:
import json

import iso8601
import pandas as pd

import modules.analyse_ecc as aecc
import modules.gtimeline_wrapper as gtwrapper


path_0 = 'derived_data/03_ecc_similarity_per_video/Cam_0.csv'
path_1 = 'derived_data/03_ecc_similarity_per_video/Cam_1.csv'
path_2 = 'derived_data/03_ecc_similarity_per_video/Cam_2.csv'
path_3 = 'derived_data/03_ecc_similarity_per_video/Cam_3.csv'
accuracy = 1.5

In [2]:
intervalls_0, excluded_0 = aecc.get_intervalls(path_0, accuracy)
intervalls_1, excluded_1 = aecc.get_intervalls(path_1, accuracy)
intervalls_2, excluded_2 = aecc.get_intervalls(path_2, accuracy)
intervalls_3, excluded_3 = aecc.get_intervalls(path_3, accuracy)

# Vorbereitung für das Darstellen mittels gTimeline

In [3]:
def add_tooltip(row):
    string="""
    <table style="font-size:10px">
        <tr>
            <td><b>Index:</b></td><td>{index}</td>
        </tr>
        <tr>
            <td><b>Start:</b></td> <td>{time_start}</td>
        </tr>
         <tr>
            <td><b>End:</b></td> <td>{time_end}</td>
        </tr>
        <tr>
            <td><b>Duration:</b></td><td>{duration}</td>
        </tr>
        <tr>
            <td><b>Filename Start:</b></td><td>{filename_start}</td>
        </tr>
        <tr>
            <td><b>Filename End:</b></td><td>{filename_end}</td>
        </tr>
    </table>
    """.format(index=row['bar_label'],
               time_start=row['start'],
               time_end=row['end'],
               duration=row['duration'],
               filename_start=row['start_video'],
               filename_end=row['end_video'])
    return string

In [4]:
def build_gtime_df(intervalls, label):
    
    df_inter = pd.DataFrame(data=intervalls, columns=['start_video', 'end_video'])

    df_inter['start'] = df_inter.apply(lambda row: iso8601.parse_date(row['start_video'].split('--')[0].split('_')[2]), axis = 1)
    df_inter['end'] = df_inter.apply(lambda row: iso8601.parse_date(row['end_video'].split('--')[1][:-4]), axis = 1)
    df_inter['duration'] = df_inter.apply(lambda row: row['end'] - row['start'], axis=1)

    df_inter['bar_label']=df_inter.index.values

    df_inter['row_label'] = label

    df_inter['tooltip'] = df_inter.apply(add_tooltip, axis=1)

    df_inter_export = df_inter.drop(['start_video', 'end_video', 'duration'], axis=1)
    
    return df_inter_export

In [5]:
df_Cam_0 = build_gtime_df(intervalls_0, 'Cam_0_ecc')
df_Cam_1 = build_gtime_df(intervalls_1, 'Cam_1_ecc')
df_Cam_2 = build_gtime_df(intervalls_2, 'Cam_2_ecc')
df_Cam_3 = build_gtime_df(intervalls_3, 'Cam_3_ecc')
intervals = pd.concat([df_Cam_0, df_Cam_1, df_Cam_2, df_Cam_3], axis=0, ignore_index=True)

In [6]:
intervals.to_pickle('./derived_data/03_ecc_similarity_per_video/df_ecc_intervalls.pkl')

In [7]:
g_dict = gtwrapper.create_gtimeline_dict(intervals, 'row_label', 'bar_label', 'start', 'end', 'tooltip')

with open('docs/03_get_ecc_intervals/data.json', 'w') as fp:
    json.dump(g_dict, fp)

In [8]:
!firefox -p develop docs/03_get_ecc_intervals/3_get_ecc_intervals.html

In [9]:
!jupyter nbconvert --to notebook --execute 04_overall_anaylses.ipynb

[NbConvertApp] Converting notebook 04_overall_anaylses.ipynb to notebook
[NbConvertApp] Executing notebook with kernel: python3
[NbConvertApp] Writing 2017 bytes to 04_overall_anaylses.nbconvert.ipynb
